# 1. Notebook Preparation

## 1.1 Import libraries

In [1]:
#General
import os
import glob
import numpy as np
import pandas as pd

## 1.2 Functions

In [2]:
os.chdir(r'C:\Users\Pablo\Documents\GitHub\Data_Analysis') #Copy directory where other ipynb are located
%run -i Functions.ipynb
%run -i Columns.ipynb

In [3]:
def insert_column(data, location, col_name, value):
    if col_name not in data.columns:
        data.insert(location, col_name, value)

# 2. Load files and set folder location

## 2.1 Request and retrieve logs

In [4]:
#Folder directory
print("Files Folder")
InDirectory = input(prompt=">>>")
os.chdir(InDirectory)

#Explore directory and select all files with the given format
import_extension = 'csv'
files = [i for i in sorted(glob.glob('*.{}'.format(import_extension)), key=os.path.getmtime)] #Change to *.**{} to search in subfolders

Files Folder


>>> C:\Users\Pablo\Documents\Jupyter\Jupyter\Jupyter\Results\DV\CL11\High_temp\SN169530\Tests started on  01_19_2022


### 2.1.1 Check files uploaded

In [5]:
try:
    print('Files selected:')
    for count,file in enumerate(files):
        print(count+1,'-',get_file_size(file),'MB -',file)
except:
    print('Error','The list of files to import is empty. Click on the Upload File button and select a log file')

Files selected:
1 - 5.29 MB - CL11_High_temp SN169530Ed 3.6 ULP 95_38_19 Date 01_19_2022 -  01_01_51 PM.csv
2 - 5.26 MB - CL11_High_temp SN169530Ed 3.6 ULP 95_38_19 Date 01_19_2022 -  02_02_11 PM.csv
3 - 1.5 MB - CL11_High_temp SN169530Ed 3.6 ULP 95_38_19 Date 01_19_2022 -  03_02_04 PM.csv


## 2.2 Choose the Ouput Directory

In [5]:
print("Output Folder")
OutDirectory = input(prompt=">>>")

Output Folder:  C:\Users\m0081459\Documents\Jupyter\Results\Pruebas Motilla\Log Christian\CSV y Script - Jupyter


### 2.2.1 Set and check Output Folder

In [6]:
try:
    if (OutDirectory == None):
        print('Warning','Directory not found. By default, files will be placed in Documents/_Validation/Reports')
        OutDirectory = os.path.expanduser('~/Documents')
    print('Output Directory:' + OutDirectory)
except:
    print('Error','Select a valid directory to store output files')

Output Directory:C:\Users\m0081459\Documents\Jupyter\Results\Pruebas Motilla\Log Christian\CSV y Script - Jupyter


In [7]:
# Sort files by date
import_extension = 'csv'
files = [i for i in sorted(glob.glob('*.{}'.format(import_extension)), key=os.path.getmtime)]

## 2.3 Define Test Name

In [8]:
print("Introduce Test Name")
test = input(prompt=">>>")

Introduce Test Name:  CL10


## 2.4 Create Output Subfolders

In [14]:
dirName = OutDirectory + '\Combined csvs'
dirPlot = OutDirectory + '\Plots'
simplification_rate = 5;

try:
    os.mkdir(dirName)
    print('Directory ' , dirName, ' Created ') 
except FileExistsError:
    print('Directory ' , dirName, ' already exists')
    
try:
    os.mkdir(dirPlot)
    print('Directory ' , dirPlot, ' Created ') 
except FileExistsError:
    print('Directory ' , dirPlot, ' already exists')

Directory  C:\Users\m0081459\Documents\Jupyter\Results\DV\CL11\High_temp\SN169530\Tests started on  01_19_2022\Combined csvs  already exists
Directory  C:\Users\m0081459\Documents\Jupyter\Results\DV\CL11\High_temp\SN169530\Tests started on  01_19_2022\Plots  already exists


## 2.5 Define Project

In [9]:
project = request_project(project_question)

Introduce Project
1 :  STEP1
2 :  BEV
3 :  PHEV


>>> 1


STEP1  selected.


## 2.6 Load and combine csv Files

In [11]:
#Request simplification rate
prompt = ">>>"
print("Introduce Simplification rate: ")
try:
    simplification_rate = int(input(prompt=prompt))
except ValueError:
    print("Invalid number. Simplification rate set to 10.")
    simplification_rate = 10

Introduce Simplification rate: 


>>> 20


In [18]:
df = pd.concat([pd.read_csv(f, sep=';', low_memory=False, thousands='.', decimal=',',error_bad_lines=False, lineterminator='\n') for f in files])

#Clean up the file
df = df[pd.notnull(df['Time'])] #Remove rows where the column Time contains NaN values

#If the sampling rate is to high use the simplifacation rate variable to reduce the number of data points to plot
df = df.iloc[np.arange(0,df.shape[0], simplification_rate)] #Filters only 1 out of X rows to speed up the ploting process
df['Measurement'] = np.arange(0,df.shape[0])
df = df.reset_index(drop=True)

# 3 Modify DataFrame

## 3.1 Prepare dataframe

In [27]:
#Create a column with datatime format
if 'Date_Time' not in df.columns:
    data_time = df['Date'] + " " + df['Time'];
    pd.to_datetime(data_time, errors='coerce');
    df.insert(0, 'Date_Time',data_time)

## 3.2 Define columns to plot

In [14]:
#MAIN COLUMNS
Main_labels = main_columns(df)

In [15]:
#TEMPERATURE 
Temp_labels = temp_columns(df, project[1])

In [16]:
#MEASUREMENTS FROM HV AC I/O
HVAC_labels_V = hvac_v_columns(df, project[1])
HVAC_labels_I = hvac_i_columns(df, project[1])
HVAC_labels_P = hvac_p_columns(df, project[1])

HVAC_labels = merge_dicts(HVAC_labels_V, HVAC_labels_I,HVAC_labels_P)

In [17]:
#MEASUREMENTS FROM HV DC I/O
HVDC_labels_V = hvdc_v_columns(df, project[1])
HVDC_labels_I = hvdc_i_columns(df, project[1])
HVDC_labels_P = hvdc_p_columns(df, project[1])

HVDC_labels = merge_dicts(HVDC_labels_V, HVDC_labels_I,HVDC_labels_P)

In [18]:
#MEASUREMENTS FROM LV DC I/O
LVDC_labels_V = lvdc_v_columns(df, project[1])
LVDC_labels_I = lvdc_i_columns(df, project[1])
LVDC_labels_P = lvdc_p_columns(df, project[1])

LVDC_labels = merge_dicts(LVDC_labels_V, LVDC_labels_I, LVDC_labels_P)

In [19]:
#STATUS
Status_labels = status_columns(df, project[1])

In [20]:
#DATA LOGGER
DL1_labels = dl1_columns(df)
DL2_labels = dl2_columns(df)

## 3.3 Combine labels

In [26]:
labels = merge_dicts(Main_labels, Temp_labels, HVAC_labels, HVDC_labels, LVDC_labels, Status_labels, DL1_labels, DL2_labels)
numeric_labels = merge_dicts(Temp_labels, HVAC_labels, HVDC_labels, LVDC_labels, Status_labels, DL1_labels, DL2_labels)
columns = get_keys(labels)

## 3.4 DataFrame to plot
Create an auxiliar dataframe from the original df with only the columns to plot

In [61]:
#We create a copy of the dataframe filtering the columns included in the labels dictionaries
df_simplified = df[columns].copy()

#Change Columns names to its labels
df_simplified.rename(columns=labels,inplace=True)

## 3.5 Create DataFrame for SQL server upload

## 3.5 Export to a new csv

In [161]:
#Generate a csv file combining all selected files and with the simplification rate applied
export_extension = 'csv'
try:
    df.to_csv(dirName + '\\' + test + ' '  + 'Combined CSV' + '.' + export_extension,sep=';',decimal=',',index=False)
    df_simplified.to_csv(dirName + '\\' +  test + ' '  + 'Combined CSV Simplified' + '.' + export_extension,sep=';',decimal=',',index=False)
    print('Success','File creation completed')
except:
    print('Error','File creation failed')

Success File creation completed


In [162]:
import subprocess
subprocess.Popen(r'explorer /select,{}'.format(dirName))